# **BT thực hành 5.5: Naive Bayes - Weather & Play**

> Cập nhật: **07/2023**

In [1]:
## Môi trường lập trình
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)
folder = '/content/gdrive/My Drive/Colab Notebooks/LDS3/Thuc hanh/Ch5_Probability'

Mounted at /content/gdrive


In [2]:
## Thư viện
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import seaborn           as sns

---
## **Yêu cầu:**
1. Mở tập tin dữ liệu **weather.xlsx**.
2. Tính xác suất chơi thể thao khi Weather = Sunny.
---

In [4]:
## 1. Mở tập tin dữ liệu weather.xlsx chứa 14 mẫu dữ liệu
data = pd.read_excel(folder + "/Data/NaiveBayes - weather_play.xlsx", sheet_name = 'Samples', index_col = 0)
n    = data.shape[0] # số lượng mẫu n = 14

## (Feature = 'Weather', Target = 'Play')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Weather  14 non-null     object
 1   Play     14 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes


In [5]:
## Quan sát dữ liệu
data

,Weather,Play
0,Rainy,Yes
1,Sunny,Yes
2,Overcast,Yes
3,Overcast,Yes
4,Sunny,No
5,Rainy,Yes
6,Sunny,Yes
7,Overcast,Yes
8,Rainy,No
9,Sunny,No


In [6]:
## STEP 1: Chỉnh dạng dữ liệu --> Các classes của target trở thành columns
data_crosstab = pd.crosstab(data['Weather'], data['Play'], margins = False)
print(data_crosstab)

Play      No  Yes
Weather          
Overcast   0    5
Rainy      2    2
Sunny      2    3


In [7]:
## Thêm cột tổng cộng 'Total'
data_crosstab["Total"] = data_crosstab["No"] + data_crosstab["Yes"]
print(data_crosstab)

Play      No  Yes  Total
Weather                 
Overcast   0    5      5
Rainy      2    2      4
Sunny      2    3      5


In [8]:
## STEP 2:
## a) Tính các xác suất P(X): P(Overcast), P(Rainy), P(Sunny)
data_crosstab["P_X"] = data_crosstab["Total"] / n
print(data_crosstab)

Play      No  Yes  Total       P_X
Weather                           
Overcast   0    5      5  0.357143
Rainy      2    2      4  0.285714
Sunny      2    3      5  0.357143


In [9]:
## b) Tính các xác suất của classes P(c): P(No), P(Yes)
Total_No  = sum(data_crosstab["No"].values)
P_No      = Total_No / n # Có 4 mẫu c = No trong tổng số 14 mẫu
print(f'P(No)  = {P_No}')

Total_Yes = sum(data_crosstab["Yes"].values)
P_Yes     = Total_Yes / n # Có 10 mẫu c = Yes trong tổng số 14 mẫu
print(f'P(Yes) = {P_Yes}')

P(No)  = 0.2857142857142857
P(Yes) = 0.7142857142857143


In [10]:
## c) Tính các xác xuất P(X | c): P(X | No), P(X | Yes)
##    P(Overcast | No)  = 0 / 4  = 0
##    P(Rainy    | No)  = 2 / 4  = 0.5
##    P(Sunny    | No)  = 2 / 4  = 0.5
##    P(Overcast | Yes) = 5 / 10 = 0.5
##    P(Rainy    | Yes) = 2 / 10 = 0.2
##    P(Sunny    | Yes) = 3 / 10 = 0.3
data_crosstab["P_No_X"]  = data_crosstab["No"]  / Total_No
data_crosstab["P_Yes_X"] = data_crosstab["Yes"] / Total_Yes
print(data_crosstab)

Play      No  Yes  Total       P_X  P_No_X  P_Yes_X
Weather                                            
Overcast   0    5      5  0.357143     0.0      0.5
Rainy      2    2      4  0.285714     0.5      0.2
Sunny      2    3      5  0.357143     0.5      0.3


In [11]:
## 2. Tính xác suất chơi thể thao khi Weather = Sunny.
def prediction_Yes(data, event_name):
    # Rút trích dòng dữ liệu tương ứng ([event_name]) trong likelihood table
    event = data.filter(items = [event_name], axis = 0)
    print('Event', event)

    P_Yes_event = event["P_Yes_X"]*(sum(data["Yes"].values)/sum(data["Total"].values))/event["P_X"]
    return P_Yes_event

In [12]:
P_Yes_Sunny = prediction_Yes(data_crosstab, "Sunny")
print('==> Prediction: Play = Yes' if (P_Yes_Sunny.values[0] > 0.5) else 'Play = No')

Event Play     No  Yes  Total       P_X  P_No_X  P_Yes_X
Weather                                           
Sunny     2    3      5  0.357143     0.5      0.3
==> Prediction: Play = Yes
